# Load Libarary

In [6]:
from main_c import main, getTrainedModel
import os
from tqdm import tqdm
from pdb2fasta import pdb2fasta
import pickle as pkl
import pipeline
print(1)

1


In [3]:
import pandas as pd
dti_train = pd.read_csv('~/Pandeng/PD/data/Davis/split_data/train.csv')
dti_val = pd.read_csv('~/Pandeng/PD/data/Davis/split_data/test.csv')
dti_test = pd.read_csv('~/Pandeng/PD/data/Davis/split_data/valid.csv')

all_proteins = pd.read_csv('~/Pandeng/PD/data/Davis/all_proteins.csv')
print(1)

1


In [14]:
all_proteins

,name,sequence
0,AAK1,MKKFFDSRREQGGSGLGSGSSGGGGSTSGLGSGYIGRVFGIGRQQV...
1,ABL2,MVLGTVLLPPNSYGRDQDTSLCCLCTEASESALPDLTDHFASCVED...
2,ACVR1,MVDGVMILPVLIMIALPSPSMEDEKPKVNPKLYMCVCEGLSCGNED...
3,ACVR1B,MAESAGASSFFPLVVLLLAGSGGSGPRGVQALLCACTSCLQANYTC...
4,ACVR2A,MGAAAKLAFAVFLISCSSGAILGRSETQECLFFNANWEKDRTNQTG...
...,...,...
356,PRKG2,MGNGSVKPKHSKHPDGHSGNLTTDALRNKVTELERELRRKDAEIQE...
357,PRKR,MAGDLSAGFFMEELNTYRQKQGVVLKYQELPNSGPPHDRRFTFQVI...
358,PRKX,MEAPGLAQAAAAESDSRKVAEETPDGAPALCPSPEALSPEPPVYSL...
359,PRP4,MAAAETQSLREQPEMEDANSEKSINEENGEVSEDQSQNKHSRHKKK...


In [ ]:
import pickle as pkl
import os
import numpy as np
with open('Davis/name_to_seq.pkl', 'rb') as f:
    name_to_seq = pkl.load(f)

# Max Sequence Cutoff, For Cuda Memory
# We use Tesla V100, the max Sequence length is 800~1000

In [7]:
# seq, name = all_proteins['sequence'], all_proteins['name']
names = name_to_seq.keys()
seqs = name_to_seq.values()
from tqdm import tqdm
save = './Davis/cut_seqs/'
# l = []
# for i in tqdm(range(len(seqs))):
for n in names:
    s = name_to_seq[n]
    # n = names[i]
    path = save + n + '.seq'
    if len(s) < 801:
        with open(path, 'w') as f:
            f.write(f'>{n}\n')
            f.write(s)
    else:
        s_list = [s[i:i+800] for i in range(0, len(s), 800)]
        assert s == ''.join(s_list)
        for j in range(len(s_list)):
            path = save + n + f'_{j}.seq'
            with open(path, 'w') as f:
                f.write(f'>{n}_{j}\n')
                f.write(s_list[j])
print('done')

done


# Feature Generation

In [8]:
input_dir = './Davis/cut_seqs'
output_dir = './Davis/omega_2'
model = None
already = os.listdir(output_dir)
already = [key.split('.')[0] for key in already]
### 如果输入是目录下的所有序列
if os.path.isdir(input_dir):
    if not os.path.exists(output_dir):
        os.system('mkdir '+output_dir)
    for f in tqdm(os.listdir(input_dir)):
        print(f)
        # if f.split('.')[0] in already:
        #     continue
        if f[-3:] != 'seq':continue
        if model == None:
            model, args, state_dict, forward_config = getTrainedModel(input_dir+'/'+f, output_dir)
            model = model.to('cuda:0')
#         try:
#             main(model, input_dir+'/'+f, output_dir, args, state_dict, forward_config)
#         except Exception as e:
#             print('failed:', f, e)
            
        args.input_file = input_dir+'/'+f
        main(model, input_dir+'/'+f, output_dir, args, state_dict, forward_config)
else:
    args.input_file = input_dir+'/'+input_dir
    main(model, input_dir+'/'+input_dir, output_dir, args, state_dict, forward_config)

  0%|          | 0/522 [00:00<?, ?it/s]

FLT3_0.seq
INFO:root:Loading weights from ./pretrained_model/model.pt
INFO:root:Constructing OmegaFold
INFO:root:Reading ./Davis/cut_seqs/FLT3_0.seq
MPALARDGGQLPLLVVFSAMIFGTITNQDLPVIKCVLINHKNNDSSVGKSSSYPMVSESPEDLGCALRPQSSGTVYEAAAVEVDVSASITLQVLVDAPGNISCLWVFKHSSLNCQPHFDLQNRGVVSMVILKMTETQAGEYLLFIQSEATNYTILFTVSIRNTLLYTLRRPYFRKMENQDALVCISESVPEPIVEWVLCDSQGESCKEESPAVVKKEEKVLHELFGTDIRCCARNELGRECTRLFTIDLNQTPQTTLPQLFLKVGEPLWIRCKAVHVNHGFGLTWELENKALEEGNYFEMSTYSTNRTMIRILFAFVSSVARNDTGYYTCSSSKHPSQSALVTIVEKGFINATNSSEDYEIDQYEEFCFSVRFKAYPQIRCTWTFSRKSFPCEQKGLDNGYSISKFCNHKHQPGEYIFHAENDDAQFTKMFTLNIRRKPQVLAEASASQASCFSDGYPLPSWTWKKCSDKSPNCTEEITEGVWNRKANRKVFGQWVSSSTLNMSEAIKGFLVKCCAYNSLGTSCETILLNSPGPFPFIQDNISFYATIGVCLLFIVVLTLLICHKYKKQFRYESQLQMVQVTGSSDNEYFYVDFREYEYDLKWEFPRENLEFGKVLGSGAFGKVMNATAYGISKTGVSIQVAVKMLKEKADSSEREALMSELKMMTQLGSHENIVNLLGACTLSGPIYLIFEYCCYGDLLNYLRSKREKFHRTWTEIFKEHNFSFYPTFQSHPNSSMPGSREVQIHPDSDQISGLHGNSFHSEDEIEYENQKRLEEEEDLNVLTFEDLLCFAYQVAKGME
INFO:root:Predicting 1th chain in ./Davis/cut_seqs

  0%|          | 1/522 [01:13<10:38:06, 73.49s/it]

TRKC_0.seq
INFO:root:Reading ./Davis/cut_seqs/TRKC_0.seq
MDVSLCPAKCSFWRIFLLGSVWLDYVGSVLACPANCVCSKTEINCRRPDDGNLFPLLEGQDSGNSNGNASINITDISRNITSIHIENWRSLHTLNAVDMELYTGLQKLTIKNSGLRSIQPRAFAKNPHLRYINLSSNRLTTLSWQLFQTLSLRELQLEQNFFNCSCDIRWMQLWQEQGEAKLNSQNLYCINADGSQLPLFRMNISQCDLPEISVSHVNLTVREGDNAVITCNGSGSPLPDVDWIVTGLQSINTHQTNLNWTNVHAINLTLVNVTSEDNGFTLTCIAENVVGMSNASVALTVYYPPRVVSLEEPELRLEHCIEFVVRGNPPPTLHWLHNGQPLRESKIIHVEYYQEGEISEGCLLFNKPTHYNNGNYTLIAKNPLGTANQTINGHFLKEPFPESTDNFILFDEVSPTPPITVTHKPEEDTFGVSIAVGLAAFACVLLVVLFVMINKYGRRSKFGMKGPVAVISGEEDSASPLHHINHGITTPSSLDAGPDTVVIGMTRIPVIENPQYFRQGHNCHKPDTYVQHIKRRDIVLKRELGEGAFGKVFLAECYNLSPTKDKMLVAVKALKDPTLAARKDFQREAELLTNLQHEHIVKFYGVCGDGDPLIMVFEYMKHGDLNKFLRAHGPDAMILVDGQPRQAKGELGLSQMLHIASQIASGMVYLASQHFVHRDLATRNCLVGANLLVKIGDFGMSRDVYSTDYYRLFNPSGNDFCIWCEVGGHTMLPIRWMPPESIMYRKFTTESDVWSFGVILWEIFTYGKQPWFQLSNTEVIECITQGRVLERPRVCPKEVY
INFO:root:Predicting 1th chain in ./Davis/cut_seqs/TRKC_0.seq
INFO:root:800 residues in this chain.
INFO:root:Finished prediction in 56.99 sec

  0%|          | 2/522 [02:11<9:17:19, 64.31s/it] 

...

100%|█████████▉| 520/522 [4:20:14<01:09, 34.77s/it]

PIK4CB_0.seq
INFO:root:Reading ./Davis/cut_seqs/PIK4CB_0.seq
MGDTVVEPAPLKPTSEPTSGPPGNNGGSLLSVITEGVGELSVIDPEVAQKACQEVLEKVKLLHGGVAVSSRGTPLELVNGDGVDSEIRCLDDPPAQIREEEDEMGAAVASGTAKGARRRRQNNSAKQSWLLRLFESKLFDISMAISYLYNSKEPGVQAYIGNRLFCFRNEDVDFYLPQLLNMYIHMDEDVGDAIKPYIVHRCRQSINFSLQCALLLGAYSSDMHISTQRHSRGTKLRKLILSDELKPAHRKRELPSLSPAPDTGLSPSKRTHQRSKSDATASISLSSNLKRTASNPKVENEDEELSSSTESIDNSFSSPVRLAPEREFIKSLMAIGKRLATLPTKEQKTQRLISELSLLNHKLPARVWLPTAGFDHHVVRVPHTQAVVLNSKDKAPYLIYVEVLECENFDTTSVPARIPENRIRSTRSVENLPECGITHEQRAGSFSTVPNYDNDDEAWSVDDIGELQVELPEVHTNSCDNISQFSVDSITSQESKEPVFIAAGDIRRRLSEQLAHTPTAFKRDPEDPSAVALKEPWQEKVRRIREGSPYGHLPNWRLLSVIVKCGDDLRQELLAFQVLKQLQSIWEQERVPLWIKPYKILVISADSGMIEPVVNAVSIHQVKKQSQLSLLDYFLQEHGSYTTEAFLSAQRNFVQSCAGYCLVCYLLQVKDRHNGNILLDAEGHIIHIDFGFILSSSPRNLGFETSAFKLTTEFVDVMGGLDGDMFNYYKMLMLQGLIAARKHMDKVVQIVEIMQQGSQLPCFHGSSTIRNLKERFHMSMTEEQLQLLVEQMVDGSMRSI
INFO:root:Predicting 1th chain in ./Davis/cut_seqs/PIK4CB_0.seq
INFO:root:800 residues in this chain.
INFO:root:Finished prediction in 57.

100%|█████████▉| 521/522 [4:21:12<00:41, 41.71s/it]

PIK3C2B_2.seq
INFO:root:Reading ./Davis/cut_seqs/PIK3C2B_2.seq
NVLLGEVNIRLRELDLAQEKTGWFALGSRSHGTL
INFO:root:Predicting 1th chain in ./Davis/cut_seqs/PIK3C2B_2.seq
INFO:root:34 residues in this chain.
INFO:root:Finished prediction in 0.48 seconds.
INFO:root:Saving prediction to ./Davis/omega_2/PIK3C2B_2.pdb
INFO:root:Saved
INFO:root:Done!


100%|██████████| 522/522 [4:21:12<00:00, 30.02s/it]


In [3]:
with open('./results/demo_feature/2ahm 6m0j_w_gly|Folding@home-2ahm|Folding@home-6m0j_w_gly.pdb.pkl','rb') as f:
    demo_re = pkl.load(f)
demo_re.keys()

dict_keys(['struct_node', 'struct_edge'])

In [4]:
demo_re['struct_node'].shape, demo_re['struct_edge'].shape

((194, 384), (194, 194, 128))

In [6]:
demo_re['struct_node']

array([[ 3.7012e+00,  1.7529e+00,  1.7559e+00, ..., -1.4660e+03,
        -2.5109e+01, -4.1938e+01],
       [ 7.3320e+00, -2.5469e+01, -1.6980e-01, ..., -1.4570e+03,
         3.8062e+01, -5.0719e+01],
       [-2.9727e+00,  3.5344e+01,  1.3516e+01, ..., -1.7870e+03,
         6.1781e+01, -4.8828e+00],
       ...,
       [-1.1211e+01,  2.7859e+01, -1.1260e+00, ..., -1.4540e+03,
         2.5969e+01, -9.0156e+00],
       [-4.7594e+01, -3.1781e+01,  5.4188e+01, ..., -1.1640e+03,
        -4.7266e+00, -1.6969e+01],
       [-1.6469e+01, -1.5492e+01,  2.3469e+01, ..., -1.6540e+03,
        -1.4320e+01, -1.5922e+01]], dtype=float16)